In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

# Monthly Data

In [36]:
#Read Data
Exportaciones_netas = pd.read_csv("../csv_results/Exportaciones_Mensuales_Netas.csv")
Incidentes_geopoliticos = pd.read_csv('../csv_results/incidentes_geopoliticos.csv')
Terrorismo = pd.read_csv("../csv_results/Terrorism.csv")

Variaciones_crudo = pd.read_csv("../csv_results/Variaciones_Crudo_Mensual")
Variaciones_NGPL = pd.read_csv('../csv_results/Variaciones_NGPL_Mensual')
Variaciones_Others = pd.read_csv('../csv_results/Variaciones_Others_Mensual')
Variaciones_Refinado = pd.read_csv('../csv_results/Variaciones_Refinery_Mensual')

#Delete unnecesary columns in the variaciones Data set
del Variaciones_crudo['Unnamed: 0']
del Variaciones_NGPL['Unnamed: 0']
del Variaciones_Others['Unnamed: 0']
del Variaciones_Refinado['Unnamed: 0']

#Delete unnecesary columns in the Incidentes geopoliticos DB
del Incidentes_geopoliticos["Unnamed: 0"]

#Delete unnecesary columns in the Expotaciones_netas DB
Exportaciones_netas = Exportaciones_netas.drop(columns=["Unnamed: 0", "Exp_notes", 'Imp_notes'])

#Delete unnecesary columns in the Terrorismo DB
#Terrorismo = Terrorismo.drop(columns=['Unnamed: 0', 'imonth', 'approxdate', 'targtype1', 'targsubtype1', 'doubtterr', 'alternative','alternative_txt'])


In [42]:
Terrorismo['alternative_txt'].unique().tolist()

[nan,
 'Intra/Inter-group Conflict',
 'Other Crime Type',
 'Insurgency/Guerilla Action',
 'Lack of Intentionality',
 'State Actors']

In [35]:
Terrorismo.head()

,iyear,iday,extended,resolution,country_txt,region_txt,targtype1_txt,targsubtype1_txt,multiple,success,latitude,longitude,motive,period
0,1972,1,0,NaN,Kuwait,Middle East & North Africa,Business,Gas/Oil/Electric,0,1,NaN,NaN,NaN,1972-01
1,1972,14,0,NaN,Iran,Middle East & North Africa,Business,Gas/Oil/Electric,0,0,35.724533,51.405190,NaN,1972-08
2,1972,23,0,NaN,Iran,Middle East & North Africa,Utilities,Oil,0,0,30.345997,48.287136,NaN,1972-08
3,1973,24,0,NaN,Venezuela,South America,Utilities,Oil,0,1,NaN,NaN,NaN,1973-11
4,1973,6,1,1974-04-20,Argentina,South America,Business,Gas/Oil/Electric,0,1,-34.617680,-58.444435,NaN,1973-12


In [7]:
#Undo pivot tables
Variaciones_crudo = Variaciones_crudo.set_index("Country").stack().reset_index()
Variaciones_NGPL = Variaciones_NGPL.set_index("Country").stack().reset_index()
Variaciones_Others = Variaciones_Others.set_index("Country").stack().reset_index()
Variaciones_Refinado = Variaciones_Refinado.set_index("Country").stack().reset_index()

In [21]:
###Create a list with definitive countries names 
#Problematic contries names are marked with a variant in parenthesis
countries = ['Albania','Algeria','Angola','Argentina','Azerbaijan','Benin','Bolivia','Brazil','Brunei','Cameroon','Canada','China','Colombia','Republic of the Congo', 'Democratic Republic of the Congo', 'Denmark','Ecuador','Egypt','Estonia','Gabon','Georgia','Ghana','Guatemala','Indonesia','Iran','Iraq','Kazakhstan','Kuwait','Kyrgyzstan','Libya','Malaysia','Mexico','Mongolia', 'Myanmar (Burma)','Nigeria','Norway','Oman','Qatar','Russia (Soviet Union)','Saudi Arabia','South Sudan','Sudan','Trinidad and Tobago','Tunisia','Turkmenistan','United Arab Emirates','United Kingdom','Venezuela','Vietnam','Yemen (North Yemen)']

#### Create a dictionary of countries names who may be replaced
#First create a dictionary with countries in each db
db_countries = { "var" : Variaciones_crudo["Country"].unique().tolist(),
                "exp" : Exportaciones_netas['Country'].unique().tolist(),
                "inc" : Incidentes_geopoliticos['country'].unique().tolist(),
                "ter" : Terrorismo['country_txt'].unique().tolist() }

#Then for each db_list check which countries do not match with our list and put them in a dictionary
to_change = {'var' : [x for x in db_countries["var"] if x not in countries], 
                    'exp' : [x for x in db_countries["exp"] if x not in countries],
                    'inc' : [x for x in db_countries["inc"] if x not in countries],
                    'ter' : [x for x in db_countries["ter"] if x not in countries] }

#Print the dictionary in order to find which names need to be replaced
to_change

{'var': ['Burma', 'Congo-Brazzaville', 'Congo-Kinshasa', 'Russia', 'Yemen'],
 'exp': ['Brunei Darussalam',
  'Bolivia, Plurinational State of',
  'Iran, Islamic Republic of',
  'Myanmar',
  'Russian Federation',
  'Venezuela, Bolivarian Republic of',
  'Viet Nam',
  'Yemen'],
 'inc': ['Congo', 'DR Congo (Zaire)'],
 'ter': ['Myanmar', 'Yemen', 'Russia']}

In [22]:
#Now replace the names in each db

#Exportaciones netas
Exportaciones_netas["Country"] = Exportaciones_netas["Country"].replace({'Brunei Darussalam' : 'Brunei',
  'Bolivia, Plurinational State of' : 'Bolivia',
  'Iran, Islamic Republic of' : 'Iran' ,
  'Myanmar' : 'Myanmar (Burma)',
  'Russian Federation' : 'Russia (Soviet Union)',
  'Venezuela, Bolivarian Republic of' : 'Venezuela',
  'Viet Nam' : 'Vietnam',
  'Yemen' : 'Yemen (North Yemen)'}) 

#Incidentes geopoliticos
Incidentes_geopoliticos["country"] = Incidentes_geopoliticos["country"].replace({'Congo' : 'Republic of the Congo', 'DR Congo (Zaire)' : 'Democratic Republic of the Congo'})

#Terrorismo
Terrorismo["country_txt"] = Terrorismo["country_txt"].replace({'Myanmar' : 'Myanmar (Burma)', 'Yemen' : 'Yemen (North Yemen)' , 'Russia' : 'Russia (Soviet Union)', 'South Vietnam' : 'Vietnam' })

#Variaciones en el valor 
Variaciones_crudo["Country"] = Variaciones_crudo["Country"].replace({'Burma' : 'Myanmar (Burma)', 'Congo-Brazzaville' : 'Republic of the Congo', 'Congo-Kinshasa' : 'Democratic Republic of the Congo', 'Russia' : 'Russia (Soviet Union)' , 'Yemen' : 'Yemen (North Yemen)'})
Variaciones_NGPL["Country"] = Variaciones_NGPL["Country"].replace({'Burma' : 'Myanmar (Burma)', 'Congo-Brazzaville' : 'Republic of the Congo', 'Congo-Kinshasa' : 'Democratic Republic of the Congo', 'Russia' : 'Russia (Soviet Union)' , 'Yemen' : 'Yemen (North Yemen)'})
Variaciones_Others["Country"] = Variaciones_Others["Country"].replace({'Burma' : 'Myanmar (Burma)', 'Congo-Brazzaville' : 'Republic of the Congo', 'Congo-Kinshasa' : 'Democratic Republic of the Congo', 'Russia' : 'Russia (Soviet Union)' , 'Yemen' : 'Yemen (North Yemen)'})
Variaciones_Refinado["Country"] = Variaciones_Refinado["Country"].replace({'Burma' : 'Myanmar (Burma)', 'Congo-Brazzaville' : 'Republic of the Congo', 'Congo-Kinshasa' : 'Democratic Republic of the Congo', 'Russia' : 'Russia (Soviet Union)' , 'Yemen' : 'Yemen (North Yemen)'})


In [25]:
#Now lets check lenght of periods of each database

print( f' exp: {len(Exportaciones_netas)}, inc: {len(Incidentes_geopoliticos)}, Ter: {len(Terrorismo)}, var_crud: {len(Variaciones_crudo)}, var_ngp: {len(Variaciones_NGPL)}, var_otr: {len(Variaciones_Others)}, var_ref: {len(Variaciones_Refinado)}')

 exp: 9102, inc: 52989, Ter: 1605, var_crud: 42400, var_ngp: 32800, var_otr: 32800, var_ref: 32800


Next spets:


- Check lenght on period or month of each database 

# Yearly Data